**Import packages and load data**

In [1]:
import pandas as pd
import gc
gc.collect()

0

In [ ]:
customer_behavior = pd.read_csv("download.csv")

**Inspect and clean product_category, sessionQuality, transactions and adjust price**

In [3]:
customer_behavior.head()

,Unnamed: 0,users,visitNumber,visitId,date,startTime,continent,subcontinent,country,region,...,sessionQuality,totals_revenue,transactions,brand,price,impression,click,product_name,medium,source
0,0,8098018953492717341,1,1491017061,2017-03-31,2017-04-01 03:24:21,Asia,Southeast Asia,Indonesia,not available in demo dataset,...,NaN,0,NaN,(not set),21990000,1,0,Category,organic,google
1,1,1757329975873508385,1,1490423257,2017-03-24,2017-03-25 06:27:37,Americas,Northern America,United States,not available in demo dataset,...,NaN,0,NaN,(not set),14990000,1,0,Category,organic,google
2,2,5951153772896753075,1,1490414492,2017-03-24,2017-03-25 04:01:32,Americas,Northern America,United States,not available in demo dataset,...,NaN,0,NaN,(not set),24990000,1,0,Category,organic,google
3,3,1728401204856015969,1,1490389094,2017-03-24,2017-03-24 20:58:14,Americas,Northern America,United States,District of Columbia,...,NaN,0,NaN,(not set),1500000,1,0,Category,organic,google
4,4,1728401204856015969,1,1490389094,2017-03-24,2017-03-24 20:58:14,Americas,Northern America,United States,District of Columbia,...,NaN,0,NaN,(not set),24990000,1,0,Category,organic,google


In [4]:
customer_behavior = customer_behavior.drop(customer_behavior.columns[0], axis=1)

In [5]:
customer_behavior.columns

Index(['users', 'visitNumber', 'visitId', 'date', 'startTime', 'continent',
       'subcontinent', 'country', 'region', 'action', 'transactionId',
       'transactionRevenue', 'product_category', 'timeOnSite', 'pageviews',
       'numHits', 'sessionQuality', 'totals_revenue', 'transactions', 'brand',
       'price', 'impression', 'click', 'product_name', 'medium', 'source'],
      dtype='object')

In [6]:
customer_behavior.head()
# sessionQuality NA change to -1, transactions NA change to 0

,users,visitNumber,visitId,date,startTime,continent,subcontinent,country,region,action,...,sessionQuality,totals_revenue,transactions,brand,price,impression,click,product_name,medium,source
0,8098018953492717341,1,1491017061,2017-03-31,2017-04-01 03:24:21,Asia,Southeast Asia,Indonesia,not available in demo dataset,0,...,NaN,0,NaN,(not set),21990000,1,0,Category,organic,google
1,1757329975873508385,1,1490423257,2017-03-24,2017-03-25 06:27:37,Americas,Northern America,United States,not available in demo dataset,0,...,NaN,0,NaN,(not set),14990000,1,0,Category,organic,google
2,5951153772896753075,1,1490414492,2017-03-24,2017-03-25 04:01:32,Americas,Northern America,United States,not available in demo dataset,0,...,NaN,0,NaN,(not set),24990000,1,0,Category,organic,google
3,1728401204856015969,1,1490389094,2017-03-24,2017-03-24 20:58:14,Americas,Northern America,United States,District of Columbia,0,...,NaN,0,NaN,(not set),1500000,1,0,Category,organic,google
4,1728401204856015969,1,1490389094,2017-03-24,2017-03-24 20:58:14,Americas,Northern America,United States,District of Columbia,0,...,NaN,0,NaN,(not set),24990000,1,0,Category,organic,google


In [7]:
def clean_elementary(category):
    # Remove unwanted characters and trim whitespace
    category = category.replace('${escCatTitle}', '') \
                       .replace('${productitem.product.origCatName}', '') \
                       .replace('(not set)', '')
    # Remove trailing slashes
    if category.endswith('/'):
        category = category[:-1]  # Remove the last character (the slash)

    # Remove prefix 'Home/'
    if category.startswith('Home/'):
        category = category.replace('Home/', '', 1)  # Remove 'Home/' only once
    
    if category.startswith('/'):
        category = category.replace('/', '', 1)

    return category

In [8]:
customer_behavior['clean_intermediate'] = customer_behavior['product_category'].apply(clean_elementary)

In [9]:
exact_replacements = {
    "Apparel//Men's-T-Shirts": "Apparel/Men's/Men's-T-Shirts",
    'Backpacks': 'Bags/Backpacks',
    'Bottles': 'Accessories/Drinkware/Water Bottles and Tumblers',
    'Drinkware/Bottles': 'Accessories/Drinkware/Water Bottles and Tumblers',
    'Drinkware': 'Accessories/Drinkware',
    'Drinkware/Mugs and Cups': 'Accessories/Drinkware/Mugs and Cups',
    'Drinkware/Water Bottles and Tumblers': 'Accessories/Drinkware/Water Bottles and Tumblers',
    'Electronics/Accessories/Drinkware': 'Accessories/Drinkware',
    'Drinkware/Mugs': 'Accessories/Drinkware/Mugs and Cups',
    'Mugs': 'Accessories/Drinkware/Mugs and Cups',
    'Clearance Sale': 'Sale/Clearance',
    'Spring Sale!': 'Sale/Spring',
    'Fun': 'Accessories/Fun',
    'Fruit Games': 'Accessories/Fun',
    'Lifestyle/Fun': 'Accessories/Fun',
    "Men's-Outerwear": "Apparel/Men's/Men's-Outerwear",
    "Men's/Men's-Performance Wear": "Apparel/Men's/Men's-Performance Wear",
    'Mens Outerwear': "Apparel/Men's/Men's-Outerwear",
    'More Bags': 'Bags/More Bags',
    'Notebooks & Journals': 'Office/Notebooks & Journals',
    'Office/Office Other': 'Office/Other',
    'Office/Writing Instruments': 'Office/Writing',
    'Shop by Brand': 'Brands',
    'Shop by Brand/Google': 'Brands/Google',
    'Shop by Brand/Waze': 'Brands/Waze',
    'Shop by Brand/YouTube': 'Brands/YouTube',
    'Shop by Brand/Android': 'Brands/Android',
    'Google': 'Brands/Google',
    'Housewares': 'Accessories/Housewares',
    'Headgear': 'Apparel/Headgear',
    'Headwear': 'Apparel/Headwear',
    'Home': '',
    'Tumblers': 'Accessories/Drinkware/Water Bottles and Tumblers',
    'Waze': 'Brands/Waze',
    'Wearables': 'Apparel',
    "Wearables/Men's T-Shirts": "Apparel/Men's/Men's-T-Shirts",
    'Writing': 'Office/Writing',
    'YouTube': 'Brands/Youtube',
    'Android': 'Brands/Android',
}

In [10]:
customer_behavior['cleaned_category'] = customer_behavior['clean_intermediate'].replace(exact_replacements)

In [11]:
customer_behavior[['main_category', 'sub_category', 'subsub_category']] = customer_behavior['cleaned_category'].str.split('/', expand=True)

In [12]:
customer_behavior.drop(columns=['product_category','clean_intermediate', 'cleaned_category'], inplace=True)

In [ ]:
columns_to_fill = ['main_category', 'sub_category', 'subsub_category']

customer_behavior[columns_to_fill] = customer_behavior[columns_to_fill].fillna('Other')

In [13]:
customer_behavior['totals_revenue'] = customer_behavior['totals_revenue'] / 1e6
customer_behavior['transactionRevenue'] = customer_behavior['transactionRevenue'] / 1e6
customer_behavior['price'] = customer_behavior['price'] / 1e6

In [14]:
customer_behavior = customer_behavior.assign(
    sessionQuality=customer_behavior['sessionQuality'].fillna(-1),
    transactions=customer_behavior['transactions'].fillna(0)
)

In [15]:
customer_behavior.head()

,users,visitNumber,visitId,date,startTime,continent,subcontinent,country,region,action,...,brand,price,impression,click,product_name,medium,source,main_category,sub_category,subsub_category
0,8098018953492717341,1,1491017061,2017-03-31,2017-04-01 03:24:21,Asia,Southeast Asia,Indonesia,not available in demo dataset,0,...,(not set),21.99,1,0,Category,organic,google,Brands,YouTube,None
1,1757329975873508385,1,1490423257,2017-03-24,2017-03-25 06:27:37,Americas,Northern America,United States,not available in demo dataset,0,...,(not set),14.99,1,0,Category,organic,google,Brands,YouTube,None
2,5951153772896753075,1,1490414492,2017-03-24,2017-03-25 04:01:32,Americas,Northern America,United States,not available in demo dataset,0,...,(not set),24.99,1,0,Category,organic,google,Apparel,Headgear,None
3,1728401204856015969,1,1490389094,2017-03-24,2017-03-24 20:58:14,Americas,Northern America,United States,District of Columbia,0,...,(not set),1.50,1,0,Category,organic,google,Accessories,None,None
4,1728401204856015969,1,1490389094,2017-03-24,2017-03-24 20:58:14,Americas,Northern America,United States,District of Columbia,0,...,(not set),24.99,1,0,Category,organic,google,Apparel,Men's,Men's-T-Shirts


**Export cleaned data**

In [16]:
customer_behavior.to_csv('cleaned_customer_behavior.csv', index=False)

**Import into Google Colab, convert date and startTime to date time objects. Not that timeOnSite is in seconds**